In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from os import path
import os

sys.path.append(os.path.abspath("../utils"))
sys.path.append(os.path.abspath("../models"))

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
from sklearn.decomposition import PCA
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from simple_nn import *
from model_utils import *
from simple_nn_utils import *


from torch.optim.lr_scheduler import MultiStepLR

In [4]:
INPUT_DIM = 20
N_CLASSES = 5

In [5]:
# setting device on GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
fake_data, fake_targets = make_multilabel_classification(
    n_samples=100,
    n_features=INPUT_DIM,
    n_classes=N_CLASSES,
    allow_unlabeled=False,
    random_state=314,
)

In [7]:
indices = range(fake_data.shape[0])
X_train, X_test, y_train, y_test, idxs_train, idxs_test = train_test_split(
    fake_data,
    fake_targets,
    indices,
    #                                                     stratify=train_test_stratify, # don't know how this works with multilabel
    test_size=0.2,
    random_state=314,
)
# X_train = torch.tensor([[1,3,5],[50,100,10]])
# y_train = torch.tensor([[0,1],[1,1]])
# X_test = torch.tensor([[1,3,5],[50,100,10]])
# y_test = torch.tensor([[0,1],[1,1]])

In [8]:
X_train, X_test, y_train, y_test

(array([[1., 4., 1., ..., 1., 2., 0.],
        [2., 2., 0., ..., 0., 1., 2.],
        [3., 3., 1., ..., 6., 3., 3.],
        ...,
        [4., 6., 5., ..., 0., 3., 0.],
        [4., 5., 4., ..., 0., 0., 3.],
        [4., 2., 4., ..., 0., 1., 1.]]),
 array([[ 5.,  3.,  3.,  4.,  3.,  5.,  2.,  5.,  4.,  4.,  0.,  1.,  1.,
          3.,  3.,  1.,  4.,  0.,  3.,  0.],
        [ 2.,  2.,  1.,  3.,  0.,  2.,  0.,  3.,  3.,  2.,  1.,  3.,  4.,
          5.,  2.,  4.,  3.,  2.,  2.,  2.],
        [ 1.,  4.,  2.,  4.,  0.,  1.,  2.,  3.,  3.,  1.,  2.,  0.,  7.,
          1.,  2.,  4.,  3.,  1.,  3.,  1.],
        [ 1.,  3.,  5.,  4.,  0.,  0.,  5.,  2.,  1.,  3.,  2.,  0.,  2.,
          3.,  3.,  4.,  3.,  2.,  3.,  3.],
        [ 3.,  2.,  5.,  0.,  4.,  4.,  4.,  1.,  1.,  5.,  4.,  1.,  1.,
         11.,  2.,  1.,  4.,  1.,  5.,  1.],
        [ 3.,  3.,  1.,  6.,  2.,  2.,  2.,  1.,  2.,  2.,  3.,  1.,  3.,
          4.,  1.,  1.,  2.,  3.,  1.,  5.],
        [ 3.,  1.,  3.,  1.,  4.,  0.

In [9]:
train_dataset = TensorDataset(
    torch.tensor(X_train).float(), torch.tensor(y_train).float()
)
test_dataset = TensorDataset(torch.tensor(X_test).float(), torch.tensor(y_test).float())

train_loader = DataLoader(
    train_dataset,
    batch_size=5,
    #  collate_fn=collate_wrapper,
    pin_memory=True,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=5,
    #  collate_fn=collate_wrapper,
    pin_memory=True,
)

In [22]:
config = {
    # how often do we want to do evaluation
    "eval_freq": 2,
    "learning_rate": 0.005,
    "lr_weight_decay": 0.001,
    "lr_scheduler": True,
    "epochs": 10,
    "batch_size": 8,
    "dropout": 0.1,
    "focal_loss_gamma": 5.0,
    "layer_size": 64,
}

In [27]:
model = AbdPainPredictionMLP(
    INPUT_DIM, N_CLASSES, layer_size=config["layer_size"], dropout=config["dropout"]
).to(device)
print(model, flush=True)

opt = torch.optim.Adam(
    model.parameters(),
    lr=config["learning_rate"],
    weight_decay=config["lr_weight_decay"],
)  # lr=1e-5)

# opt = torch.optim.SGD(model.parameters(), lr=config['learning_rate'])
# weight_decay = config['lr_weight_decay'])# lr=1e-5)

if config["lr_scheduler"]:
    scheduler = MultiStepLR(
        opt, milestones=list(range(0, config["epochs"], 100))[1:], gamma=0.5
    )
else:
    scheduler = None

# opt = torch.optim.RMSprop(model.parameters(), lr=0.001)
loss = MultilabelFocalLoss(N_CLASSES, gamma=config["focal_loss_gamma"])
# loss = torch.nn.BCEWithLogitsLoss()#reduction="sum")


multilabel_train(
    model,
    loss,
    opt,
    train_loader,
    test_loader,
    device,
    n_epochs=config["epochs"],
    scheduler=scheduler,
)

# y_pred = model(test_features)
# top_k_accuracy_score(y_test, y_pred, labels=le.classes_, k=10)

AbdPainPredictionMLP(
  (fc1): Linear(in_features=20, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=5, bias=True)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
in train - pred:
 tensor([[1., 1., 0., 1., 0.],
        [0., 0., 1., 0., 0.],
        [1., 1., 0., 1., 0.],
        [1., 0., 1., 0., 1.],
        [0., 1., 1., 0., 0.]], device='cuda:0')
in train - out:
 tensor([[0.5179, 0.6781, 0.3393, 0.5114, 0.3898],
        [0.4293, 0.3330, 0.5116, 0.4452, 0.3476],
        [0.5441, 0.6890, 0.4150, 0.6663, 0.4639],
        [0.5313, 0.3342, 0.5844, 0.4052, 0.8103],
        [0.4774, 0.5044,

/gpfs/milgram/project/rtaylor/vs428/conda_envs/factcheck_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/milgram/project/rtaylor/vs428/conda_envs/factcheck_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


in train - pred:
 tensor([[0., 1., 0., 0., 0.],
        [1., 0., 0., 1., 0.],
        [1., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 1., 1.]], device='cuda:0')
in train - out:
 tensor([[0.4008, 0.5161, 0.3687, 0.4381, 0.4775],
        [0.5692, 0.4945, 0.4203, 0.5191, 0.4583],
        [0.5308, 0.4782, 0.4462, 0.6395, 0.4719],
        [0.5366, 0.4696, 0.4111, 0.4023, 0.4774],
        [0.5063, 0.4127, 0.4318, 0.5392, 0.5120]], device='cuda:0',
       grad_fn=<SigmoidBackward>)
in train - target:
 tensor([[0., 1., 0., 0., 1.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [1., 1., 0., 1., 1.]], device='cuda:0')
# correct?:
 tensor(22, device='cuda:0')
tensor(0.0751, device='cuda:0', grad_fn=<MulBackward0>) 

in train - pred:
 tensor([[0., 1., 0., 0., 0.],
        [0., 1., 0., 1., 1.],
        [0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1.]], device='cuda:0')
in train - out:
 tenso

/gpfs/milgram/project/rtaylor/vs428/conda_envs/factcheck_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/milgram/project/rtaylor/vs428/conda_envs/factcheck_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/milgram/project/rtaylor/vs428/conda_envs/factcheck_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(

In [12]:
# Testing seems to work on a synthetic dataset